# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Hyndai').getOrCreate()

     |████████████████████████████████| 204.7MB 65kB/s 
     |████████████████████████████████| 204kB 40.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=e4907f969ba1f80c9f0290aaafaff964a2ab5e9015a759e0aaa8e620880540ac
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [41]:
from pyspark.ml.regression import LinearRegression

In [5]:
df = spark.read.csv('/content/cruise_ship_info.csv', inferSchema = True, header = True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [48]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
Cruise_line_indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = Cruise_line_indexer.fit(df).transform(df)
#encoder = OneHotEncoder(inputCols=["Cruise_line"], outputCols=["Cruise_lie_vec"])

assembler = VectorAssembler(inputCols= ['cruise_cat','Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density'], outputCol = 'features')

In [50]:
output = assembler.transform(indexed)

In [51]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[16.0,6.0,30.2769...|
|[16.0,6.0,30.2769...|
|[1.0,26.0,47.262,...|
|[1.0,11.0,110.0,2...|
|[1.0,17.0,101.353...|
|[1.0,22.0,70.367,...|
|[1.0,15.0,70.367,...|
|[1.0,23.0,70.367,...|
|[1.0,19.0,70.367,...|
|[1.0,6.0,110.2389...|
|[1.0,10.0,110.0,2...|
|[1.0,28.0,46.052,...|
|[1.0,18.0,70.367,...|
|[1.0,17.0,70.367,...|
|[1.0,11.0,86.0,21...|
|[1.0,8.0,110.0,29...|
|[1.0,9.0,88.5,21....|
|[1.0,15.0,70.367,...|
|[1.0,12.0,88.5,21...|
|[1.0,20.0,70.367,...|
+--------------------+
only showing top 20 rows



In [52]:
final_data = output.select('features', 'Crew')

In [53]:
final_data.show()

+--------------------+----+
|            features|Crew|
+--------------------+----+
|[16.0,6.0,30.2769...|3.55|
|[16.0,6.0,30.2769...|3.55|
|[1.0,26.0,47.262,...| 6.7|
|[1.0,11.0,110.0,2...|19.1|
|[1.0,17.0,101.353...|10.0|
|[1.0,22.0,70.367,...| 9.2|
|[1.0,15.0,70.367,...| 9.2|
|[1.0,23.0,70.367,...| 9.2|
|[1.0,19.0,70.367,...| 9.2|
|[1.0,6.0,110.2389...|11.5|
|[1.0,10.0,110.0,2...|11.6|
|[1.0,28.0,46.052,...| 6.6|
|[1.0,18.0,70.367,...| 9.2|
|[1.0,17.0,70.367,...| 9.2|
|[1.0,11.0,86.0,21...| 9.3|
|[1.0,8.0,110.0,29...|11.6|
|[1.0,9.0,88.5,21....|10.3|
|[1.0,15.0,70.367,...| 9.2|
|[1.0,12.0,88.5,21...| 9.3|
|[1.0,20.0,70.367,...| 9.2|
+--------------------+----+
only showing top 20 rows



In [54]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [55]:
train_data.describe().show()

+-------+------------------+
|summary|              Crew|
+-------+------------------+
|  count|               107|
|   mean| 7.656261682242996|
| stddev|3.5935675035505454|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [56]:
test_data.describe().show()

+-------+------------------+
|summary|              Crew|
+-------+------------------+
|  count|                51|
|   mean| 8.083529411764705|
| stddev|3.3224188920299693|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [57]:
lr = LinearRegression(labelCol = 'Crew')

In [58]:
lr_model = lr.fit(train_data)

In [59]:
test_results = lr_model.evaluate(test_data)

In [60]:
test_results.rootMeanSquaredError

0.6526946843980382

In [61]:
test_results.r2

0.9606348828615143

In [62]:
test_results.meanSquaredError

0.4260103510414548

In [63]:
test_results.meanAbsoluteError

0.5347085093831552

In [64]:
from pyspark.sql.functions import corr

In [65]:
df.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

